# kaggle 房价预测

## 读取数据

In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

In [7]:
# 读取训练数据集和测试数据集
train_data=pd.read_csv("../data/kaggle_house/train.csv")
test_data=pd.read_csv("../data/kaggle_house/test.csv")


In [4]:
# 将train_data中的label抽取出来,同时在train data中删除label,删除掉不需要的label标签
sale_price=train_data["SalePrice"]
train_data.drop(columns=["Id","SalePrice"],inplace=True)

In [8]:
train_data.shape

(1460, 81)

In [10]:
# 查看特定的行，以及特定的列,pandas相关操作
train_data.iloc[0:4,[0, 1, 2, 3, -3, -2, -1]]

,Id,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,WD,Normal,208500
1,2,20,RL,80.0,WD,Normal,181500
2,3,60,RL,68.0,WD,Normal,223500
3,4,70,RL,60.0,WD,Abnorml,140000


In [11]:
# 去掉id 标签
all_features = pd.concat((train_data.iloc[:, 1:-1], test_data.iloc[:, 1:]))

# 数据预处理
简单处理，可以直接将数值的特征进行归一化操作

In [12]:
# 取出 数值特征的列,并且进行标准化
numeric_features=all_features.dtypes[all_features.dtypes!=object].index
all_features[numeric_features]=all_features[numeric_features].apply(lambda x:(x-x.mean())/(x.std()))

In [13]:
# 缺失值采用均值来进行填充
all_features[numeric_features]=all_features[numeric_features].fillna(0)

In [14]:
# 将非数值形的特征转换为one-hot的形式
# dummy_na=True将缺失值也当作合法的特征值并为其创建指示特征
all_features = pd.get_dummies(all_features, dummy_na=True)
all_features.shape 

(2919, 331)

In [15]:
# 数据转换为tensor
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype=torch.float)
test_features = torch.tensor(all_features[n_train:].values, dtype=torch.float)
train_labels = torch.tensor(train_data.SalePrice.values, dtype=torch.float).view(-1, 1)

# 模型设计与训练

In [18]:
# 损失函数定义
criterion=nn.MSELoss()

In [39]:
class Net(nn.Module):
    def __init__(self,feature_num,output_num):
        super(Net,self).__init__()
        self.linear=nn.Linear(feature_num,output_num)
        # 初始化
        for model in self.modules():
            self.__weight_init(model)
    def forward(self,x):
        return self.linear(x)
    # 初始化层的函数
    def __weight_init(self,model):
        if isinstance(model, nn.Linear):
            nn.init.xavier_normal_(model.weight)
            nn.init.constant_(model.bias, 0)
        # 也可以判断是否为conv2d，使用相应的初始化方式 
        elif isinstance(model,nn.Conv2d):
            nn.init.kaiming_normal_(model.weight, mode='fan_out', nonlinearity='relu')
         # 是否为批归一化层
        elif isinstance(model, nn.BatchNorm2d):
            nn.init.constant_(model.weight, 1)
            nn.init.constant_(model.bias, 0)

In [40]:
def get_net(feature_num):
    net = Net(feature_num, 1)
    return net

In [41]:
def log_rmse(net, features, labels):
    with torch.no_grad():
        # 将小于1的值设成1，使得取对数时数值更稳定
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(criterion(clipped_preds.log(), labels.log()))
    return rmse.item()

In [42]:
def train(net, train_features, train_labels, test_features, test_labels,
          num_epochs, learning_rate, weight_decay, batch_size):
    train_ls, test_ls = [], []
    # 构建成为torch 能够使用的dataset的形式, feature-> dataSet -> dataLoader
    dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_iter = torch.utils.data.DataLoader(dataset, batch_size, shuffle=True)
    optimizer = torch.optim.Adam(params=net.parameters(), lr=learning_rate, weight_decay=weight_decay) 
    net = net
    for epoch in range(num_epochs):
        for X, y in train_iter:
            loss = criterion(net(X), y.float())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_ls.append(log_rmse(net, train_features, train_labels))
        if test_labels is not None:
            test_ls.append(log_rmse(net, test_features, test_labels))
    return train_ls, test_ls

# 深度学习中可以使用 机器学习中的交叉验证
数据不多的时候，可以使用交叉验证来分析，解决过拟合和欠拟合的问题

In [43]:
# 返回第 i 次 交叉验证所需要的数据
def get_k_fold_data(k,i,X,y):
    assert k>1
    # 将数据分成多少等份
    fold_size=X.shape[0]//k
    X_train,y_train=None,None
    for j in range(k):
        idx=slice(j*fold_size,(j+1)*fold_size)
        X_part,y_part=X[idx,:],y[idx]
        # 取得第i次作为验证集
        if j==i:
            X_valid,y_valid=X_part,y_part
        # 初次进行赋值
        elif X_train is None:
            X_train,y_train=X_part,y_part
        else:
            X_train=torch.cat((X_train,X_part),dim=0)
            y_train = torch.cat((y_train, y_part), dim=0)
    return X_train, y_train, X_valid, y_valid

In [49]:
# 训练K次并返回训练和验证的平均误差。
import sys
sys.path.append("..")
from utils import *
def k_fold(k, X_train, y_train, num_epochs,learning_rate, weight_decay, batch_size):
    train_loss_sum,valid_loss_sum=0,0
    for i in range(k):
        X_train, y_train, X_valid, y_valid=get_k_fold_data(k,i,X_train,y_train)
        # 初始化net
        net = get_net(X_train.shape[1])
        train_loss,valid_loss=train(net,X_train, y_train, X_valid, y_valid,learning_rate=learning_rate,
                                    num_epochs=num_epochs,weight_decay=weight_decay,batch_size=batch_size)
        train_loss_sum+=train_loss[-1]
        valid_loss_sum+=valid_loss[-1]
        if i==0:
            semilogy(range(1, num_epochs + 1), train_loss, 'epochs', 'rmse',
                         range(1, num_epochs + 1), valid_loss,['train', 'valid'])
        print('fold %d, train rmse %f, valid rmse %f' % (i, train_loss[-1], valid_loss[-1]))
    return train_loss_sum / k, valid_loss_sum / k

In [ ]:
k, num_epochs, lr, weight_decay, batch_size = 5, 100, 5, 0, 64
train_l, valid_l = k_fold(k, train_features, train_labels, num_epochs, lr, weight_decay, batch_size)
print('%d-fold validation: avg train rmse %f, avg valid rmse %f' % (k, train_l, valid_l))

In [1]:
# 测试，将得到的最好的参数进行保存
def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = get_net(train_features.shape[1])
    train_ls, _ = train(net, train_features, train_labels, None, None,
                        num_epochs, lr, weight_decay, batch_size)
    semilogy(range(1, num_epochs + 1), train_ls, 'epochs', 'rmse')
    print('train rmse %f' % train_ls[-1])
    preds = net(test_features).detach().numpy()
    return preds